In [40]:
import os
import json
import glob
import pprint
pp = pprint.PrettyPrinter(indent=2)

In [2]:
data_root = 'multiwoz/HDSA'

In [3]:
glob.glob(data_root + '/*')

['multiwoz/HDSA/results.txt.pred.non_delex',
 'multiwoz/HDSA/results.txt.pred.BERT_dim128_w_domain.pred',
 'multiwoz/HDSA/results.txt.non_delex']

### Read ResGEN outputs

In [23]:
def vlgan_json(path):
    vlgan_predictions = {}
    with open(path) as vlganf:
        for line in vlganf:
            po = json.loads(line)
            vlgan_predictions[po['0']['info']['name']] = po
    return vlgan_predictions

In [24]:
vlgan_predictions = vlgan_json('multiwoz/VLGAN/multiwoz_predictions_Nov-21.json')

In [25]:
len(vlgan_predictions)

1000

In [26]:
vlgan_predictions

{'SNG0073.json': {'0': {'user-x': 0,
   'gold': "I would like a taxi from Saint John's college to Pizza Hut Fen Ditton.",
   'info': {'name': 'SNG0073.json'}},
  '1': {'user-x': 1,
   'gold': 'What time do you want to leave and what time do you want to arrive by?',
   'resgen': ['sure ! where will you be departing from ?']},
  '2': {'user-x': 0,
   'gold': 'I want to leave after 17:15.',
   'resgen': ['i need to arrive by <unk> .']},
  '3': {'user-x': 1,
   'gold': '\nBooking completed! your taxi will be blue honda Contact number is 07218068540',
   'resgen': ['booking completed ! booked car contact number <unk> <unk> <unk>']},
  '4': {'user-x': 0,
   'gold': 'Thank you for all the help! I appreciate it.',
   'resgen': ['thank you so much .']},
  '5': {'user-x': 1,
   'gold': 'You are welcome.  Is there anything else I can help you with today?',
   'resgen': ['you re welcome . have a great day !']},
  '6': {'user-x': 0,
   'gold': 'No, I am all set.  Have a nice day.  Bye.',
   'resgen

### Augment ResGEN json

In [41]:
hdsa_predictions = json.load(open('multiwoz/HDSA/results.txt.pred.non_delex'))

In [48]:
def augment(vlgan_preds, tag):
    # hdsa_predictions = hdsa_utterances_into_dialogs(hdsa_dd_skip_file)
    
    for i in vlgan_preds:
        hdsa_sample, vlgan_sample = hdsa_predictions[i.replace('.json', '')], vlgan_preds[i]
#         print(len(hdsa_sample), len(vlgan_sample))
#         print(hdsa_sample)
#         pp.pprint(vlgan_sample)
        # assert len(hdsa_sample) == len(vlgan_sample)
        for t in vlgan_sample:
            # HDSA only predicts at agent steps!
            _t = int(t)
            if _t%2 == 1:
                vlgan_preds[i][t]['hdsa'] = hdsa_sample[_t//2]
        
        # hdsa_sample = hdsa_sample[:len(vlgan_sample)]

#         for t in range(min(12, len(vlgan_sample))):
#             _t = str(t)
#             vlgan_preds[i][_t][f'{tag}'] = hdsa_sample[t]['Predict']
#             vlgan_preds[i][_t][f'{tag}.Target'] = hdsa_sample[t]['Target']
#     return vlgan_preds

In [49]:
augment(vlgan_predictions, 'hdsa')

In [51]:
vlgan_predictions['SNG0073.json']

{'0': {'user-x': 0,
  'gold': "I would like a taxi from Saint John's college to Pizza Hut Fen Ditton.",
  'info': {'name': 'SNG0073.json'}},
 '1': {'user-x': 1,
  'gold': 'What time do you want to leave and what time do you want to arrive by?',
  'resgen': ['sure ! where will you be departing from ?'],
  'hdsa': 'what time would you like to leave or arrive by ?'},
 '2': {'user-x': 0,
  'gold': 'I want to leave after 17:15.',
  'resgen': ['i need to arrive by <unk> .']},
 '3': {'user-x': 1,
  'gold': '\nBooking completed! your taxi will be blue honda Contact number is 07218068540',
  'resgen': ['booking completed ! booked car contact number <unk> <unk> <unk>'],
  'hdsa': 'where will you be departing from ?'},
 '4': {'user-x': 0,
  'gold': 'Thank you for all the help! I appreciate it.',
  'resgen': ['thank you so much .']},
 '5': {'user-x': 1,
  'gold': 'You are welcome.  Is there anything else I can help you with today?',
  'resgen': ['you re welcome . have a great day !'],
  'hdsa': 'y